# This is a note for [Homogenous Coordinates (Cyrill Stachniss, 2020)](https://youtu.be/MQdm0Z_gNcw)

In [ ]:
using LinearAlgebra
using Random
using Distributions

In [ ]:
struct EuclidianCoord{T<:Real}
    coord::AbstractVector{T}
end

### Homogeneous coordicates have two advantages over Euclidian:
#### 1) can represent ponit at infinity with direction,
#### 2) all projective transformations as matrix multiplication.

### Homogeneous coordicates have two properties:
#### 1) norm cannot be 0.
#### 2) multiplying a non-zero scale does not change the coordinates, hence homogeneous.

In [ ]:
struct HomogeneousCoord{T<:Real}
    coord::AbstractVector{T}
    function HomogeneousCoord(coord::AbstractVector{T}) where {T<:Real}
        @assert norm(coord) != 0 "Homogeneous coordicates cannot be all zeros."
        new{T}(coord)
    end
end

function HomogeneousCoord(p::EuclidianCoord):HomogeneousCoord
    HomogeneousCoord([p.coord; [1]])
end

function EuclidianCoord(p::HomogeneousCoord):EuclidianCoord
    EuclidianCoord(p.coord[1:end-1]./p.coord[end])
end

import Base.==
function ==(p1::HomogeneousCoord, p2::HomogeneousCoord):Bool
    if size(p1.coord) != size(p2.coord)
        return false
    end
    return p1.coord / norm(p1.coord) == p2.coord / norm(p2.coord)
end

### Test cases for HomogeneousCoord

In [ ]:
pe = EuclidianCoord([1,2,3])
ph = HomogeneousCoord(pe)
pe_back = EuclidianCoord(ph)
@assert pe.coord == pe_back.coord
@assert HomogeneousCoord([1,2,3,4]) == HomogeneousCoord([2,4,6,8])

### 2D lines and 3D planes's coefficients are also homogeneous coordinates.
#### cross product of two points gives a line.
#### cross product of two lines gives the intersection point.
P.S. Lines in n-D has 2*(n-1) degree of freedom

In [ ]:
struct Line{T<:Real}
    coeff::HomogeneousCoord{T}
end

function Line(coeff::AbstractVector{T}) where {T<:Real}
    @assert size(coeff, 1) == 3 "A 2D line can be represented with 3 coeffecients."
    Line(HomogeneousCoord(coeff))
end

# The cross product of the homogeneous coord of two points is the line pass through these two points.
function Line(p1::HomogeneousCoord, p2::HomogeneousCoord)
    return Line(cross(p1.coord, p2.coord))
end

import Base.==
function ==(l1::Line, l2::Line):Bool
    return l1.coeff ==  norm(l2.coeff)
end

In [ ]:
function on(p::HomogeneousCoord, l::Line)::Bool
    return dot(p.coord, l.coeff.coord) ≈ 0
end

# The cross product of the two lines is the homogeneous coordinate of the intersection point of these two lines.
function intersect(l1::Line, l2::Line)::HomogeneousCoord
    return HomogeneousCoord(cross(l1.coeff.coord,l2.coeff.coord))
end

### Two parallel lines intersect at a point at infinity

In [ ]:
p = intersect(Line([1,2,3]), Line([1,2,31]))
@assert p.coord[3] == 0

### All points at infinity are all on the ideal line: Line([0,0,a])
Since these are 2D points and 2D lines, you can imagine all points and lines are on the ground, and Line([0,0,a]) is the horizon.

In [ ]:
@assert on(HomogeneousCoord([1,2,0]), Line([0,0,1]))

In [ ]:
struct Plane{T<:Real}
    coeff::HomogeneousCoord{T}
end

function Plane(coeff::AbstractVector{T}) where {T<:Real}
    @assert size(coeff, 1) == 4 "A 3D place can be represented with 4 coeffecients."
    Plane(HomogeneousCoord(coeff))
end

import Base.==
function ==(p1::Plane, p2::Plane):Bool
    return p1.coeff  == p2.coeff
end

In [ ]:
function on(p::HomogeneousCoord, plane::Plane)::Bool
    return dot(p.coord, plane.coeff.coord) ≈ 0
end

### All points at infinity are all on the same plane: Plane([0,0,0,a])
Imagine this as the sky

In [ ]:
@assert on(HomogeneousCoord([1,2,3,0]), Plane([0,0,0,1]))

### Transformation
- projective: 8/15 dof (2d/3d)
- affine: 6/12 dof
    - parallel line preserving
    - angle may change
- simimarity: 4/7 dof
    - angle preserving
    - size may change
- rigid body/motion: 3/6 dof
    - size and shape preserving
    - translation: 2/3 dof
    - rotation: 1/3 dof

In [ ]:
struct Transformation{T<:Real}
    matrix::AbstractMatrix{T}
    function Transformation(matrix::AbstractMatrix{T}) where {T<:Real}
        @assert matrix[end, end] == 1 "The lower right corner has to be 1."
        new{T}(matrix)
    end
end

function Transformation(A::AbstractMatrix, t::AbstractVector, a::AbstractVector)
    # For 3d, A: 3 rot + 3 scale + 3 sheer; t: 3 trans; a: 3 projective
    Transformation([A t; transpose(a) 1])
end

In [ ]:
t = Transformation([1 2 3;3 3 3;11 1 1], [0.0,1,3], [2,3,1])
t.matrix